# <center>ZUM</center>
# <center> Projekt -- dokumentacja końcowa </center>

## <center> Jan Budziński </center>
## <center> Jarosław Nachyła </center>

Temat 14: Nienadzorowana detekcja anomalii na podstawie niepodobieństwa do grup wyznaczanych
za pomocą algorytmów grupowania. Implementacja w formie opakowania umożliwiającego użycie różnych
algorytmów grupowania dostępnych w środowisku R lub Python i różnych miar niepodobieństwa. Porów-
nanie z nienadzorowaną detekcją anomalii za pomocą algorytmów klasyfikacji jednoklasowej dostępnych
w środowisku R lub Python.

## 1.Zbiory danych

### 1.1 HTTP KDD Cup 99 dataset.

Pierwszym zbiorem danych do testowania, który chcemy wykorzystać, jest zbiór zawierający anomalie w ruchu HTTP, znany jako "HTTP KDD Cup 99 dataset". Ten zbiór danych jest używany do badania i wykrywania nietypowych zachowań w ruchu sieciowym, co jest kluczowe dla bezpieczeństwa cybernetycznego. Dostępny jest pod adresem [https://odds.cs.stonybrook.edu/http-kddcup99-dataset/](https://odds.cs.stonybrook.edu/http-kddcup99-dataset/)

| Liczba przykładów | Atryb. dyskret. | Atryb. rzecz. | Brak. atryb. | Liczba klas |
|-------------------|-----------------|---------------|--------------|-------------|
| 567479            | 0               | 3             | 0            | 1           |

Table 1: Podsumowanie zbioru http.

Wszystkie 3 atrybuty zbioru są wartościami rzeczywistymi. Zbiór nie jest wstępnie podzielony na dane treningowe i testowe.
Dystrybucja klas w zbiorze: anomalii (ang. outlier) i wartości poprawnych (ang. inlier):

| Class  | N      | N[%] |
|--------|--------|------|
| inlier | 565268 | 99.6%|
| outlier| 2211   | 0.4% |

Jak widać klasy są bardzo niezbalansowane, ilość anomalii to zaledwie 0.4 procenta. Jest to duża dysproporcja, ale jest to dość popularne w tego typu zbiorach.

### 1.2 Statlog Shuttle dataset.

Zbiór danych "Shuttle" (często nazywany także "Statlog Shuttle Dataset") to zbiór używany w analizie danych i uczeniu maszynowym, który zawiera informacje na temat symulatora promu kosmicznego. Dostępny jest pod linkiem [https://odds.cs.stonybrook.edu/shuttle-dataset/](https://odds.cs.stonybrook.edu/shuttle-dataset/).

| Liczba przykładów | Atryb. dyskret. | Atryb. rzecz. | Brak. atryb. | Liczba klas |
|-------------------|-----------------|---------------|--------------|-------------|
| 49097             | 0               | 9             | 0            | 1           |

Table 2: Podsumowanie zbioru shuttle.

Wszystkie 9 atrybutów zbioru są wartościami rzeczywistymi. Zbiór nie jest wstępnie podzielony na dane treningowe i testowe.
Dystrybucja klas w zbiorze: anomalii (ang. outlier) i wartości poprawnych (ang. inlier):

| Class  | N      | N[%] |
|--------|--------|------|
| inlier | 45586  | 93%  |
| outlier| 3511   | 7%   |

Ilość anomalii jest dziesięciokrotnie większa niż dla zbioru poprzedniego. Został wybrany w celach porównawczych.

## 2. Założenia dla algorytmów grupowania i klasyfikacji jednoklasowej.

### 2.1 Trenowanie i ewaluacja na zbiorze treningowym dla algorytmów grupowania.

W eksperymentach nie korzystano z metody `predict`, ponieważ metoda ta wymaga centrów klastrów, które są dostępne tylko w przypadku algorytmu KMeans korzystającego z API. W przypadku DBSCAN wyliczenie centrów z danych może być błędem, ponieważ klastry mogą nie być wypukłe, co może prowadzić do sytuacji, w której centrum klastrów znajduje się w innym klastrze.

### 2.2 Klasyfikacja jednoklasowa - trening

W eksperymentach zastosowano algorytmy klasyfikacji jednoklasowej, takie jak One-Class SVM i Isolation Forest, które są przystosowane do detekcji anomalii. Tutaj zastosowano podział na dane treningowe jednak poniewaz mamy doczynienia z modelami jednoklasowymi to uczono model tylko na inlierach natomiast testowano na inlierach i outlierach z zachowaniem proporcji.

### 2.3 Algorytmy grupowania metryka AUC PR.

Algorytmy grupowania nie  posiadaja funkcji predict proba, w celu obliczenia tej metryki zastosowano przekształcenie dystansu punktów od centrów klastrów. Nie można jednak tego porównywac z implementacją predict proba w modelach klasyfikacji takich jak regresja logistyczna. Wyniki uzyskane w ten sposób musza podlegać krytycznej analizie. Jedynie dla KMeans można uznać, że wyniki są w 100% miarodajne, poniewaz centra klastrów są dostępne natywnie poprzez API metody.


## 2. Implementacja

### 2.1. Nienadzorowana detekcja anomalii

W ramach projektu zaimplementowano klasę <i>AnomalyDetector</i>, będącą opakowaniem na algorytmy grupowania KMeans, DBSCAN oraz AgglomerativeClustering, jak również na miary niepodobieństwa euklidesową, Mahalanobisa i Manhattan. Celem tejże klasy jest stworzenie łatwego w obsłudze środowiska testowego, które w miarę możliwości (niektóre algorytmy różnią się wejściami) będzie jednorodne w użyciu dla każdego z porównywanych algorytmów i miar.
Klasa ta zawiera następujące metody:

- fit -- dopasowuje model do danych wejściowych,
- fit_predict -- wykonuje trening modelu i jednocześnie przewidująca klasy i określająca dystanse od centrów klastrów dla danych wejściowych,
- transform_distances -- określa, czy dane wejściowe są anomaliami na podstawie dystansu otrzymanego z wybranej miary niepodobieństwa,
- transform_labels -- dla liczby klastrów większych niż 2, metoda ta zmienia przypisanie do wszystkich klastrów poza najliczniejszym w anomalię.


Dzięki tym metodom użytkownik jest w stanie w łatwy sposób testować różne modele i miary niepodobieństwa, zmieniając wyłącznie jeden parametr w kodzie.

Kod implementujący tę klasę jest w pliku anomaly_detector.py.

Ponadto utworzono klasę _AnomalyDetectorEvaluator_ zawierającą metody obliczające metryk potrzebnych do ewaluacji wytrenowanych modeli. 

Testowane metryki to:

- dokładność (accuracy)
- precyzja (precision)
- czułość (recall)
- f1 (F1 Score)
- czułość klasy pozytywnej (positive recall)
- czułość klasy negatywnej  (negative recall)
- precyzja klasy pozytywnej (positive precision)
- precyzja klasy negatywnej (negative precision)
- pole pod wykresem krzywej precyzji i czułości (AUC PRC)

Wszystkie te metryki zostały dokładniej opisane w dokumentacji wstępnej. Jednakże, z uwagi na fakt, iż dane z założenia są wysoce niezbalansowane (jako że są to dane anomalii, to wejścia o pozytywnej klasie stanowią poniżej 1% wszystkich) uznano, że zwykła dokładność może niewiele powiedzieć, jako że przypisanie wszystkim danym klasy negatywnej pozwala osiągnąć powyżej 99% poprawnych predykcji. W tym celu dodano metryki czułości i precyzji dla obu klas. Dodatkowo, z uwagi na fakt, iż dane są niezbalansowane, zdecydowano się na użycie AUC PRC, jako że jest to miara, która nie zależy od proporcji klas.

Eksperymenty były zapisywane do plików JSON, a następnie wczytywane do DataFrame'ów w celu analizy wyników. Znajdują sie one w katalogu results.
Kod z klasą AnomalyDetectorEvaluator znajduje się w pliku metrics.py.

### 2.2. Klasyfikacja jednoklasowa

W ramach projektu zaimplementowano klasę <i>OneClassClassifier</i>, będącą opakowaniem na algorytmy klasyfikacji jednoklasowej One-Class SVM oraz Isolation Forest. Implementacja znajduje sie w pliku one_class_classifier.py. Znajduje się tam klasa OneClassAnnomalyDetector. Modele są zaincjalizowane ze deaultowymi parametrami.Celem projektu nie był tuning parametrów modeli, a jedynie porównanie ich wydajności w detekcji anomalii.

## 3. Porównanie modeli grupowaniaz modelami klasyfikacji jednoklasowej.



### 3.1 Opis eksperymentów.

Nienadzorowana detekcja anomalii na podstawie niepodobieństwa do grup wyznaczanych za pomocą algorytmów grupowania została przeprowadzona w ramach eksperymentów porównujących algorytmy klasyfikacji jednoklasowej, takie jak One-Class SVM i Isolation Forest, z algorytmami grupowania. Eksperymenty miały na celu zbadanie wydajności tych modeli w kontekście detekcji anomalii.

W ramach pierwszego eksperymentu wykonano testy porównawcze iloczynu kartezjańskiego 3 wybranych modeli, 3 miar niepodobieństwa i 2 zbiorów danych, co dało łącznie 18 pojedynczych testów.

Podczas przeprowadzania eksperymentu wystąpiły pewne problemy. Mianowicie, algorytm Agglomerative Clustering cechuje się wysokimi złożonościami: złożoność obliczeniowa to \(O(n^2)\), a czasowa to \(O(n^3)\). Z tego względu trening na całych zbiorach danych, a w szczególności na zbiorze HTTP, był niemożliwy. W związku z tym zastosowano subsampling.


### 3.2 Szczegóły Subsampling i Eksperymentów

W przeprowadzonych eksperymentach dotyczących nienadzorowanej detekcji anomalii przy użyciu algorytmów grupowania, zastosowano różne techniki subsamplingu w celu obsługi dużych zbiorów danych oraz złożoności obliczeniowej niektórych algorytmów. Poniżej znajduje się szczegółowe wyjaśnienie strategii subsamplingu użytych dla różnych eksperymentów oraz wyjaśnienie, dlaczego niektóre eksperymenty nie zostały wykonane.





| Zbiór Danych      | Algorytm                    | Subsampling                           |
|-------------------|-----------------------------|---------------------------------------|
| HTTP              | Isolation Forest            | 40% zbioru danych             |
| HTTP              | One-Class SVM               | 40% zbioru danych              |
| HTTP              | DBSCAN                      | 10% zbioru danych               |
| HTTP              | Agglomerative Clustering    | 5% zbioru danych                      |

Subsampling stosowany był z zachowaniem proporcji klasy mniejszościowej tak aby próba zawierała wszystkie outliery których było odpowiedznio mało
około 0.4% dla zbioru HTTP i 7% dla zbioru shuttle. 

### 3.3 Braki w eksperymentach.

DBSCAN z metryką Mahalanobis nie zostały wykonane ze względu na bardzo długie czasy nauki nawet dla małych próbek danych(mozliwy problem w implemntacji w bibliotece).



In [ ]:
from experiments import Experiments

# w celu uruchomienia wszystkcich eksperymentów należy odkomentować poniższy kod czas wykonania około 1h
exps = Experiments()

# exps.run_http_one_class()
# exps.run_shuttle_one_class()
#
# exps.run_http_meta_cost()
# exps.run_shuttle_meta_cost()
#
# exps.run_http_dbscan_experiment()
# exps.run_http_kmeans_experiment()
# exps.run_http_agglomerative_experiment()
#
# exps.run_shuttle_dbscan_experiment()
# exps.run_shuttle_kmeans_experiment()
# exps.run_shuttle_agglomerative_experiment()



Poniżej znajduje się kod, który wczytuje wyniki eksperymentów zapisane w plikach JSON do DataFrame'ów i wyświetla wyniki.

In [37]:
import json
import os

import pandas as pd

pd.set_option('display.max_rows', 100)

def load_metrics_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data
def process_files(directory, pdataset = "http", models_excluded = []):
    results = []

    # Przeglądanie wszystkich plików w katalogu
    for file_name in os.listdir(directory):
        if file_name.endswith(".json"):
            # Parsing nazwy pliku
            parts = file_name.split('_')
            model = parts[0]
            dataset = parts[1]

            if dataset != pdataset or model in models_excluded:
                continue



            file_path = os.path.join(directory, file_name)
            metrics_data = load_metrics_from_json(file_path)



            if isinstance(metrics_data, dict):  # Dla isolation forest i svm
                metric = "N/A"
                avg_metrics = metrics_data['avg_metrics']
                result = {
                    'model': model,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'f1': avg_metrics.get('f1'),
                   
                    'auc_score': avg_metrics.get('auc_pr')
                }
                results.append(result)
            else:  # Dla pozostałych modeli
                for entry in metrics_data:
                    metric = entry['metric']
                    avg_metrics = entry['avg_metrics']
                    result = {
                    'model': model,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'f1': avg_metrics.get('f1'),
                    'auc_score': avg_metrics.get('auc_pr')
                    }
                    results.append(result)

    # Tworzenie DataFrame z wynikami
    df = pd.DataFrame(results)
    return df
pd.set_option('display.max_rows', 50)
process_files('./results/', models_excluded=['metacost'])

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,isolationforest,N/A,0.907282,1.000000,0.876376,0.729462,1.000000,0.843571,NaN
1,dbscan,euclidean,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
2,dbscan,manhattan,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
3,svm,N/A,0.623021,1.000000,0.497362,0.398738,1.000000,0.570139,NaN
4,kmeans,euclidean,0.566687,0.005427,0.568883,0.000049,0.993208,0.000098,0.538963
5,kmeans,manhattan,0.990925,0.014925,0.994742,0.010982,0.996142,0.012653,0.538963
6,kmeans,mahalanobis,0.567260,0.005427,0.569458,0.000049,0.993215,0.000098,0.539007
7,agglomerative,euclidean,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.061553
8,agglomerative,manhattan,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.063167
9,agglomerative,mahalanobis,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.062629


Wyniki dla zbioru HTTP. Analizując metrykę F1 najlepsze wyniki uzyskał model agglomerative clustering, a także DBSCAN, różnice pomiędzy nimi jak i ich wariantami metryk nie są istotne statystycznie. Jednocześnie modele te mają bardzo niską wartość AUC PR. Może to oznaczać, że model jest dobrze skalibrowany dla jednej konkretnej wartości progu, ale jego zdolność do generalizacji i stabilności przy innych progach jest ograniczona. Pamietamy także, że metryki AUC PR są oparte o dystanse od centrów klastrów co niekoniecznie może być miarodajne. Szczególnie jeśli klaster nie jest wypukły jego środek wcale nie jest środkiem wyznaczonych punktów. Widac także, że wyniki dla KMeans jesli chodzi o metryke AUC PR wyglądają bardziej wiarygodnie niż dla pozostałych metod grupowania(dla Kmeans centra klastrów są dostepne natywnie poprzez API metody). Dla modelu kmeans czułość jest bardzo niska co oznacza, że model prawie nie wykrywa outlierów. Jeślichodzi o modele klasyfikacji jednoklasowej do zdecydowanie lepszy wynik uzyskał model Isolation Forest. Być może tuning parametrów mógłby znacznie poprawić wyniki dla modelu One-Class SVM i Isolation forest. Trzy modele uzyskały 100% wynik detekcji anomalii(positive recall), co jest zaskakująco wysoką wartością. Tak więc biorac pod uwagę poprzdnia wartość F1, faworytem staje sie model DbScan z metryką Manhattan.

In [41]:
process_files('./results/', models_excluded=['metacost'], pdataset='shuttle')

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,dbscan,euclidean,0.993971,0.958416,0.996710,0.957326,0.996797,0.957871,0.215972
1,dbscan,manhattan,0.995376,0.982056,0.996402,0.954596,0.998615,0.968131,0.216555
2,svm,N/A,0.621618,0.997152,0.496440,0.397615,0.998091,0.568529,NaN
3,agglomerative,euclidean,0.995988,0.944745,0.999934,0.999096,0.995762,0.971161,0.435803
4,agglomerative,manhattan,0.995988,0.944745,0.999934,0.999096,0.995762,0.971161,0.530860
5,agglomerative,mahalanobis,0.995988,0.944745,0.999934,0.999096,0.995762,0.971161,0.429757
6,kmeans,euclidean,0.974907,0.690117,0.996841,0.943903,0.976617,0.797302,0.373327
7,kmeans,manhattan,0.974907,0.690117,0.996841,0.943903,0.976617,0.797302,0.379373
8,kmeans,mahalanobis,0.786830,0.017089,0.846115,0.008481,0.917876,0.011336,0.834842
9,isolationforest,N/A,0.884150,0.987183,0.849805,0.686609,0.994998,0.809908,NaN


Dla zbioru shuttle sytuacja wygląda analogicznie jak dla zbioru HTTP. Model agglomerative clustering oraz DBSCAN uzyskały najlepsze wyniki dla metryki F1, jednakże ich wyniki dla AUC PR są bardzo niskie, jednak sporo wyższe niz w przypadku zbioru HTTP. W tym eksperymencie znacznie lepiej poradziły sobie modele KMeans. Gdybysmy mieli się skupić na maksymalnej wykrwalności anomalii(Positive Recall) to najlepszym modelem jest One Class SVM(wynik 0.997), potem Isoaltion Forest(0.993). Modele klasyfiakcji jednoklasowej radzą sobie lepiej w tym przypadku niż modele grupowania.

### 4. Poprawa wyników algorytmu MetaCost opartego o KMeans, porównanie z algorytmem KMeans.

### 4.1 Algorytm MetaCost.

### 4. Poprawa wyników algorytmu MetaCost opartego o KMeans, porównanie z algorytmem KMeans.

### 4.1 Algorytm MetaCost


Algorytm MetaCost jest metodą, która przekształca klasyfikatory uczące się bezpośrednio na danych treningowych w klasyfikatory uwzględniające koszty błędów. Podstawową ideą MetaCost jest przekształcenie problemu klasyfikacji w taki sposób, aby minimalizować oczekiwany koszt błędów. Algorytm ten działa poprzez generowanie wielu próbek z oryginalnego zbioru danych, trenowanie modeli na tych próbkach, a następnie agregowanie wyników, aby uzyskać końcową klasyfikację.

Jedynym algorytmem grupowania, który można zastosować w algorytmie MetaCost, jest KMeans. Dlatego że posiada on metrykę odległości,  zbudowana na poprawnie wyznaczonych centrach klastrów.

Algorytm jest zaimplementowany w pliku metacost.py. Znajduje sie tam klasa MetaCost która zawiera dokumentację pseudokodu algorytmu.
Algorytm został zaimplementowany zgodnie z pracą naukową "MetaCost: A General Method for Making Classifiers Cost-Sensitive" autorstwa Pedro Domingosa dostępną pod adresem https://homes.cs.washington.edu/~pedrod/papers/kdd99.pdf).

### 4.2 Założenia eksperymentów

W orginalnej pulikacji znajdowały sie 2 algorytmy dla macierzy kosztów: Fixed-Interval Cost i Probability-Dependent Cost.
Pierwsza generuje macierz kosztów poprzez losowe przypisanie wartości ze stałego zakresu np. od [0, 1000] na przekątnej macierzy(poprawne klasyfukacje) oraz wartości z zakresu np. [0, 10000] dla pozostałych elementów. Celem jest modelowanie kosztów klasyfikacji, gdzie wartości na przekątnej dla błędnych klasyfikacji są statystycznie wyższe.


Druga metoda generuje macierz kosztów, biorąc pod uwagę prawdopodobieństwa wystąpienia każdej klasy w zbiorze treningowym. Wartości na przekątnej są losowane z zakresu [0, 1000], natomiast pozostałe elementy są losowane z zakresu np. [0, 2000] przemnożonego przez stosunek prawdopodobieństw klas.

Obie metody zostały zaimplementowane w pliku metacost.py, jednak przynosiły bardzo złe rezultaty. W eksperymentach zastosowano macierz kosztów opartą na dysproporcji między klasami (1 + stosunek liczby outlierów do liczby inlierów) dla zbioru HTTP oraz stały koszt równy 10 dla zbioru Shuttle.

#### Eksperymenty z MetaCost

Eksperymenty zostały przeprowadzone przy użyciu dwóch różnych zbiorów danych: HTTP i Shuttle. 


#### Uwagi
**Wpływ macierzy kosztów:**
Algorytm MetaCost jest bardzo wrażliwy na zmiany w macierzy kosztów. Dobre dopasowanie macierzy kosztów jest kluczowe dla osiągnięcia dobrych wyników, co jednak może być trudne do uzyskania.


### 4.2 Porównanie wyników

Wyniki eksperymentów przeprowadzonych dla algorytmu MetaCost oraz KMeans przedstawiono w poniższych tabelach. Warto zauważyć, że MetaCost z metryką manhattan osiągnął najlepsze wyniki, a KMeans miał znacznie gorszą wydajność w detekcji outlierów w porównaniu do MetaCost.

### Zbiór HTTP

In [38]:
process_files('./results/', models_excluded=['dbscan', 'agglomerative', 'isolationforest', 'svm'])

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,metacost,mahalanobis,0.740794,0.599367,0.741347,0.025614,0.998417,0.019925,NaN
1,metacost,euclidean,0.658134,0.605066,0.658341,0.026622,0.997898,0.017355,NaN
2,metacost,cityblock,0.569373,0.995477,0.567706,0.008935,0.999969,0.017711,NaN
3,kmeans,euclidean,0.566687,0.005427,0.568883,0.000049,0.993208,0.000098,0.538963
4,kmeans,manhattan,0.990925,0.014925,0.994742,0.010982,0.996142,0.012653,0.538963
5,kmeans,mahalanobis,0.567260,0.005427,0.569458,0.000049,0.993215,0.000098,0.539007


Dla zbioru Kmeans HTTP udąło się diametralnie  zwiekszyc wartość wykrywanych outlierów z 1-0.5% do prawie 100%(positive recall). Niestety wpływa to negatywnie na wykrywalność inlierów, więc udział przypadków Fałsz pozytywnych jest wysoki. Tylko 50% próbek niebędących anomaliamii w najlepszym przypadku została wykryta których jest przecież znacznie więcej.  

In [33]:
process_files('./results/', models_excluded=['dbscan', 'agglomerative', 'isolationforest', 'svm'], pdataset='shuttle')

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,metacost,mahalanobis,0.851531,0.970094,0.842399,0.321713,0.997274,0.483156,NaN
1,metacost,euclidean,0.851543,0.970664,0.842368,0.321730,0.997326,0.483265,NaN
2,metacost,cityblock,0.857625,0.957391,0.849941,0.329490,0.996154,0.490254,NaN
3,kmeans,euclidean,0.974907,0.690117,0.996841,0.943903,0.976617,0.797302,0.373327
4,kmeans,manhattan,0.786851,0.017374,0.846115,0.008621,0.917898,0.011524,0.845926
5,kmeans,mahalanobis,0.996089,0.946169,0.999934,0.999098,0.995871,0.971913,0.429757


Sytuacja tutaj wygląda podobnie jak dla zbioru HTTP. Aczkowlwiek wyniki dla metacost dla metryki negative recall są znacznie lepsze niż dla zbioru HTTP. Jednak nadal mamy dużą ilość fałszywych pozytywów, widac to także po stosunkowo niskiej wartości F1 w porównaniu do poprzednio badanych algorytmów. 

### 5. Podsumowanie

W ramach projektu przeprowadzono eksperymenty z użyciem różnych algorytmów grupowania oraz klasyfikacji jednoklasowej w celu detekcji anomalii w zbiorach danych HTTP oraz Shuttle. Wyniki eksperymentów pokazały, że modele klasyfikacji jednoklasowej radzą sobie nieco lepiej w detekcji samych anomalii niż modele grupowania. Jednak jako klasyfikatory obu klas wypadały znacznie gorzej. Najbardziej miarodajnym zbiorem był sosunkowo trudny zbiór anomalii HTTP (dużo przykładów i mało anomalli). Jednak dla obydwu zbiorów najlepszym wyborem pod względem wykrywania anomalii i małej ilości False positive jest DBSCAN(metryka euklidesowa lub manhattan). Ciekawym modelem jeśli chodzi o klasyfikację jednoklasową jest Isolation Forest, byc może po zasotowaniu tuningu hiperparametrów mógłby osiągnąc lepsze wyniki niż obecnie.

Jeśli chodzi o algorytm Meta Cost  to badania pokazały ze jest wstanie zmienić propocję wykrywanych klas w zależności od macierzy kosztów. Jednakże w naszym przypadku nie udało się uzyskać zadowalających wyniików, poniewąż jedyny algorytm jakie mozna było wykorzystac to KMeans  ktry miał najgorsze wyniki w porównaniu do pozostałych algorytmów grupowania.